# カバレッジ集約と公開ワークフロー
このノートブックでは、複数 CI ジョブで生成されたカバレッジ成果物をマージし、安定した最終レポート生成・しきい値検証・通知整形までを一気通貫で再現します。

## 1. 必要ツールのインストール（nyc/istanbul-lib-* と lcov-result-merger）
VS Code のターミナルで実行するか、下記セルを実行して必要な開発依存を導入します。Node が無い場合は Volta または nvm のセットアップを検討してください。

In [ ]:
# 開発依存の導入（必要に応じて）
# 実行環境によっては権限の都合で失敗する場合があります。
# 実運用中のリポでは package.json に追記して CI でインストールしてください。
import os
os.system("npm i -D nyc istanbul-lib-coverage istanbul-lib-report istanbul-reports lcov-result-merger cobertura-coverage || true")


## 2. ジョブ別カバレッジ成果物の取得と配置（artifacts/ 以下に集約）
GitHub CLI (gh) を使って最新のワークフローの artifacts をダウンロードするか、ローカル再現用にサンプル構成を作成します。

In [ ]:
# gh が使える場合のダウンロード（手元でログインが必要）
# !gh run list --workflow "Tests (Matrix)" --limit 1
# !gh run download --name "coverage-unit" --dir artifacts || true
# !gh run download --name "coverage-integration" --dir artifacts || true
# !gh run download --name "coverage-cb-rate" --dir artifacts || true
# !gh run download --name "coverage-event-metrics" --dir artifacts || true

# サンプル構成を作る（存在しなければ）
import os, json, pathlib
pathlib.Path('artifacts/job-a/coverage').mkdir(parents=True, exist_ok=True)
pathlib.Path('artifacts/job-b/coverage').mkdir(parents=True, exist_ok=True)
# ダミー coverage-final.json を配置（実運用では artifacts から取得されます）
sample_cov = {"src/example.ts": {"path": "src/example.ts", "statementMap": {"0": {"start": {"line": 1, "column": 0}, "end": {"line": 1, "column": 10}}}, "s": {"0": 1}, "branchMap": {}, "b": {}, "fnMap": {}, "f": {}}}
json.dump(sample_cov, open('artifacts/job-a/coverage/coverage-final.json','w'))
json.dump(sample_cov, open('artifacts/job-b/coverage/coverage-final.json','w'))
print('artifacts prepared')


## 3. coverage exclude の検証と適用（__tests__/helpers, mocks の除外）
vitest の設定から exclude を読み、必要なら後処理フィルタを適用します。

In [ ]:
# vitest.config.ts の exclude 参照と表示（読み取りのみ）
from pathlib import Path
conf = Path('vitest.config.ts').read_text(encoding='utf-8') if Path('vitest.config.ts').exists() else ''
print('vitest.config.ts loaded:', bool(conf))
print('\n'.join([line for line in conf.splitlines() if 'exclude' in line or '__tests__/helpers' in line or 'mocks' in line]))

# LCOV の後処理フィルタ（mocks や __tests__/helpers を除外）
def filter_lcov(input_path: str, output_path: str):
    keep = []
    skip = False
    for line in open(input_path, 'r', encoding='utf-8'):
        if line.startswith('SF:'):
            path = line[3:].strip()
            skip = ('/__tests__/helpers/' in path) or ('/mocks/' in path)
        if not skip:
            keep.append(line)
    open(output_path, 'w', encoding='utf-8').write(''.join(keep))
    return output_path
print('filter_lcov ready')


## 4. Istanbul JSON（coverage-final.json）のマージ
istanbul-lib-coverage を用いて artifacts/**/coverage-final.json をマージします。

In [ ]:
# Node スクリプトを生成して実行
from pathlib import Path
Path('scripts').mkdir(exist_ok=True)
node_code = r"""
const fs = require('fs');
const path = require('path');
const { createCoverageMap } = require('istanbul-lib-coverage');

function findJson(dir){
  const out = [];
  function walk(d){
     for (const f of fs.readdirSync(d)){
       const p = path.join(d,f);
       const s = fs.statSync(p);
       if (s.isDirectory()) walk(p);
       else if (f === 'coverage-final.json') out.push(p);
     }
  }
  if (fs.existsSync(dir)) walk(dir);
  return out;
}

const files = findJson('artifacts');
console.log('found coverage json files:', files.length);
const map = createCoverageMap({});
for (const f of files){
  try { const data = JSON.parse(fs.readFileSync(f,'utf8')); map.merge(data); }
  catch (e){ console.warn('merge failed:', f, e.message); }
}
fs.mkdirSync('coverage/merged', { recursive: true });
fs.writeFileSync('coverage/merged/coverage-final.json', JSON.stringify(map.toJSON()));
console.log('merged -> coverage/merged/coverage-final.json');
"""
Path('scripts/merge-istanbul.js').write_text(node_code, encoding='utf-8')
import os
os.system('node scripts/merge-istanbul.js')


## 5. LCOV（lcov.info）のマージ
lcov-result-merger を使うか、フォールバックとして Python 実装を用います。

In [ ]:
# lcov-result-merger
import os, glob
os.makedirs('coverage/merged', exist_ok=True)
paths = glob.glob('artifacts/**/lcov.info', recursive=True)
if paths:
    os.system('npx lcov-result-merger "artifacts/**/lcov.info" coverage/merged/lcov.info')
else:
    # フォールバック: 簡易 LCOV マージャ
    from collections import defaultdict
    def parse_lcov(text):
        files = {}
        current = None
        for line in text.splitlines():
            if line.startswith('SF:'):
                current = line[3:].strip()
                files.setdefault(current, defaultdict(int))
            elif line.startswith('DA:') and current:
                parts = line[3:].split(',')
                ln = int(parts[0]); hits = int(parts[1])
                files[current][ln] += hits
        return files
    agg = defaultdict(lambda: defaultdict(int))
    for p in glob.glob('artifacts/**/coverage/lcov.info', recursive=True):
        data = parse_lcov(open(p,'r',encoding='utf-8').read())
        for f, lines in data.items():
            for ln, hits in lines.items():
                agg[f][ln] += hits
    out = []
    for f, lines in agg.items():
        out.append(f'SF:{f}')
        for ln in sorted(lines):
            out.append(f'DA:{ln},{lines[ln]}')
        out.append('end_of_record')
    open('coverage/merged/lcov.info','w',encoding='utf-8').write('\n'.join(out))
print('lcov merged to coverage/merged/lcov.info (if any)')


## 6. マージ後レポートの生成（text-summary, lcov, cobertura, html）
istanbul-lib-report + istanbul-reports で最後のレポートを生成します。

In [ ]:
# Node スクリプトでレポート生成
from pathlib import Path
report_js = r"""
const fs = require('fs');
const libReport = require('istanbul-lib-report');
const reports = require('istanbul-reports');

const dataPath = 'coverage/merged/coverage-final.json';
if (!fs.existsSync(dataPath)){
  console.error('missing coverage json:', dataPath);
  process.exit(0);
}
const data = JSON.parse(fs.readFileSync(dataPath,'utf8'));
const libCoverage = require('istanbul-lib-coverage');
const map = libCoverage.createCoverageMap(data);
const context = libReport.createContext({ dir: 'coverage/merged' });
const reportTypes = ['text-summary', 'json-summary', 'lcov', 'cobertura', 'html'];
for (const t of reportTypes){
  reports.create(t, {}).execute(context, map);
}
console.log('reports generated under coverage/merged');
"""
Path('scripts/report-istanbul.js').write_text(report_js, encoding='utf-8')
import os
os.system('node scripts/report-istanbul.js')


## 7. しきい値チェックと終了コード制御
最終サマリから % を抽出して判定します（例: 80%）。

In [ ]:
# しきい値の検証（デフォルト 80%）
import json, sys
summary_path = 'coverage/merged/coverage-summary.json'
try:
    with open(summary_path,'r',encoding='utf-8') as f:
        s = json.load(f)['total']
        stmt = s['statements']['pct']
        lines = s['lines']['pct']
        funcs = s['functions']['pct']
        branch = s['branches']['pct']
        print(f"Statements: {stmt}%  Lines: {lines}%  Functions: {funcs}%  Branches: {branch}%")
        threshold = float(os.environ.get('COV_THRESHOLD', '80'))
        ok = all(v >= threshold for v in [stmt, lines, funcs, branch])
        print('threshold =', threshold, '=>', 'PASS' if ok else 'FAIL')
        if not ok:
            sys.exit(1)
except FileNotFoundError:
    print('coverage summary missing (ok if artifacts are dummy)')


## 8. 欠落ジョブの検出とフォールバック統合
期待ジョブの不足を検出し、警告を出しつつ既存成果物のみで継続します。

In [ ]:
# 欠落ジョブ検出
import glob
expected = {'unit','integration','cb-rate','event-metrics'}
found = set()
for p in glob.glob('artifacts/**/coverage-final.json', recursive=True):
    # 名前規約から推測（coverage-<group> の親ディレクトリ名などに合わせて調整）
    # 例: artifacts/coverage-unit/coverage/coverage-final.json
    parts = p.replace('\\','/').split('/')
    for g in expected:
        if any(g in seg for seg in parts):
            found.add(g)
missing = sorted(expected - found)
print('found:', sorted(found))
print('missing:', missing)
if missing:
    print('WARN: 欠落ジョブがあります。マージは継続しますが、夜間に再実行を推奨します。')


## 9. PR コメント/Slack 通知用メトリクスの整形
最終値を Markdown 化し、gh api や Slack Webhook に投稿します（ドライラン可）。

In [ ]:
# Markdown 整形と投稿例（ドライラン）
import json, os, subprocess
summary = {}
try:
    summary = json.load(open('coverage/merged/coverage-summary.json','r',encoding='utf-8'))['total']
except Exception:
    summary = {}

def md_block(s):
    return f"## Merged Coverage\n\n| Metric | % |\n|---|---:|\n| Statements | {s.get('statements',{}).get('pct','N/A')} |\n| Lines | {s.get('lines',{}).get('pct','N/A')} |\n| Functions | {s.get('functions',{}).get('pct','N/A')} |\n| Branches | {s.get('branches',{}).get('pct','N/A')} |\n"

body = md_block(summary)
print(body)

# gh で PR コメント（環境によってはトークン設定が必要、ここではドライラン）
if os.environ.get('GH_TOKEN'):
    print('Posting comment via gh (simulated) ...')
    # subprocess.run(['gh','pr','comment','--body', body], check=False)

# Slack Webhook
webhook = os.environ.get('SLACK_WEBHOOK_URL')
if webhook:
    import urllib.request, json as j
    data = j.dumps({"text": body}).encode('utf-8')
    req = urllib.request.Request(webhook, data=data, headers={'Content-Type': 'application/json'})
    try:
        urllib.request.urlopen(req)
    except Exception as e:
        print('Slack post failed:', e)


## 10. GitHub Actions 連携サンプル（artifact ダウンロードとマージ実行）
最小の集約ワークフロー YAML をファイルとして出力します。

In [ ]:
# coverage-merge-pages.yml を生成（既存ワークフローの簡易版）
from pathlib import Path
wf = r"""
name: Coverage Merge (Sample)

on:
  workflow_dispatch: {}

jobs:
  merge:
    runs-on: ubuntu-latest
    steps:
      - uses: actions/checkout@v4
      - uses: actions/download-artifact@v4
        with:
          path: artifacts
          pattern: coverage-*
          merge-multiple: true
      - uses: actions/setup-node@v4
        with:
          node-version: 20
      - run: |
          node -e "console.log('merging...')"
"""
Path('.github/workflows/coverage-merge-sample.yml').write_text(wf, encoding='utf-8')
print('wrote .github/workflows/coverage-merge-sample.yml')


## 11. nightly/日中 CI の分岐サンプル（LONG_TESTS と --coverage 強制）
LONG_TESTS=1 の場合のみ重いテストを実行し、常に --coverage を付与します。

In [ ]:
# 簡易分岐の疑似コード（実運用は GitHub Actions 側で実現）
import os
if os.environ.get('LONG_TESTS') == '1':
    print('Run long tests with coverage: npm run test:long -- --coverage')
else:
    print('Skip long tests (daytime). Always run with --coverage on nightly.')
